References:
- Automatic Music Score Recognition System Using Digital Image Processing (Chang et al.)
- https://github.com/acieroid/overscore

In [143]:
import os
import sys
import cv2
import logging

from itertools import chain
from string import digits

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from preprocessors import Opener, Grayscaler

from pprint import pprint

# for showing images in notebook
from matplotlib import pyplot as plt
import numpy as np

In [2]:
DATA_DIR = ['../data/package_aa', '../data/package_ab']

In [168]:
# test_path = '../data/package_aa/000051650-1_1_1/000051650-1_1_1.png'
# test_path = '../data/package_ab/110002956-1_30_1/110002956-1_30_1.png'
test_path = '../data/package_aa/000051661-1_1_1/000051661-1_1_1.png'

Generate placement coordinates from image and corresponding semantic notation

In [3]:
def get_placement_coordinates(image, tokens):
    tokens = '\t'.join(tokens)
    tokens = tokens.split('\t')
    print(tokens)
    

Get image and semantic data from dataset

In [26]:
def get_image_semantics_data(data_src):
    for src in data_src:
        for i, folder in enumerate(next(os.walk(src))[1]):
            semantic_path = os.path.join(src, folder, "{}.semantic".format(folder))
            img_path = os.path.join(src, folder, "{}.png")
            yield semantic_path, img_path

In [130]:
class DocumentWorkflow:
    def __init__(self,
                 preprocessor_pipeline,
                 force_output_process=False):
        self.preprocessors = preprocessor_pipeline
        self.force_output_process = force_output_process
    
    def __call__(self, image):
        if self.force_output_process:
            self.preprocessors.set_force_output_process()
        
        image = self.preprocessors(image)
        if self.force_output_process:
            cv2.imwrite('output/stage_1_preprocess.jpg', image)

In [149]:
class PreprocessorPipeline:
    def __init__(self):
        self.filters = []
    def set_force_output_process(self):
        for f in self.filters:
            f.output_process = True
    
    def __call__(self, image):
        for f in self.filters:
            image = f(image)
        return image
    

Main Workflow

In [235]:
class LineRemover:
    """
    Removes staff lines on an image.
    
    Params
    ------
    line_threshold_tolerance    how much difference between histogram max and current row length 
                                is allowed when determining staff lines
    image    is the image to have lines removed
    
    Returns
    ------
    Image without the lines
    """
    def __init__(self, line_threshold_tolerance=10, output_process = False):
        self.line_threshold_tolerance = line_threshold_tolerance
        self.output_process = output_process
    def __call__(self, image):
        hist = image.shape[1] - image.sum(axis=1)//256
        if self.output_process:
            output = image.copy()
            output = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            hist_output = np.zeros((image.shape[0], image.shape[1], 3)).astype(output.dtype)
            for r, r_o in zip(hist, hist_output):
                r = min(r, output.shape[1])
                r_o[:r] = [0, 0, 255]
            output = cv2.addWeighted(output, 0.8, hist_output, 0.8,0)
            cv2.imwrite('output/line_remover_horizontal_histogram.jpg', output)
        
        line_removed = image.copy()
        line_threshold = max(hist)
        for r, r_m in zip(hist, line_removed):
            if abs(line_threshold-r) < self.line_threshold_tolerance:
                r_m[:] = 255
        
        if self.output_process:
            cv2.imwrite('output/line_removed.jpg', line_removed)
            
        return line_removed

In [245]:
image = cv2.imread(test_path)

preprocessors = PreprocessorPipeline()
preprocessors.filters += [Grayscaler(),
                          LineRemover(),
                          Opener(kernel_size=5, iterations=2),
                          Opener(kernel_size=3, iterations=1)]

document_workflow = DocumentWorkflow(preprocessors, force_output_process=True)
output = document_workflow(image)